El GCPD recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta informacióñ se encuentra guarada en un archivo con el siguiente formato:

(fecha, id_caso, descripción, estado_caso, categoria, latitud, longitud)

Los posibels estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolucion.

Las fechas se encuentran en el formato YYYY-MM-DD.

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuales casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respeusta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.

b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

Resolver ambas consultas utilizando Pandas.

[Resolución de colaborador de la materia](https://gist.github.com/Roj/a19d82c1b8ebcb0198dc4773752b852a)

[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqroi7u4xe)

In [42]:
import pandas as pd
import numpy as np

In [43]:
np.random.seed(100)
num_datos = 10000

fecha_inicio = '2022-01-25'
fecha_fin = '2024-05-04'

estados_posibles = [1,2,3]

categorias = ["incendio intencional", "robo", "hurto",
              "robo de vehiculo", "mercaderia ilegal", 
              "comercio de pinguinos", "transporte de armas"]

centro_latitud = 40.744010
centro_longitud = -73.993943
radio_circulo_grados = 0.01

In [44]:
fecha_inicio = pd.to_datetime(fecha_inicio)
fecha_fin = pd.to_datetime(fecha_fin)

# Genera todas las horas posibles en ese rango de fechas
all_hours = pd.date_range(fecha_inicio, fecha_fin, freq='h')

random_dates = np.random.choice(all_hours, num_datos)

# Convierte el resultado a una serie de pandas
random_dates = pd.Series(random_dates)

random_dates.sort_values(inplace=True)

In [45]:
df = pd.DataFrame({
    'fecha': random_dates,
    'id_caso': np.arange(num_datos),
    'descripcion': ["s/d"]*num_datos,
    'estado_caso': np.concatenate((
                np.random.choice(estados_posibles, int(num_datos/2), p=[0.3, 0.6, 0.1]),
                np.random.choice(estados_posibles, int(num_datos/2), p=[0.15, 0.75, 0.1])
                )),
    'categoria': np.random.choice(categorias, num_datos),
    'latitud': centro_latitud + radio_circulo_grados * (2*np.random.rand(num_datos)-1),
    'longitud': centro_longitud + radio_circulo_grados * (2*np.random.rand(num_datos)-1)
})
df.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
8998,2022-01-25 00:00:00,0,s/d,2,robo,40.741344,-74.003243
3828,2022-01-25 09:00:00,1,s/d,3,transporte de armas,40.740767,-73.995798
8441,2022-01-25 10:00:00,2,s/d,2,comercio de pinguinos,40.753579,-74.000437
9253,2022-01-25 14:00:00,3,s/d,2,incendio intencional,40.748401,-73.993990
3311,2022-01-25 17:00:00,4,s/d,2,robo de vehiculo,40.743804,-73.992243


In [46]:
df.reset_index(drop=True, inplace=True)

In [47]:
df

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2022-01-25 00:00:00,0,s/d,2,robo,40.741344,-74.003243
1,2022-01-25 09:00:00,1,s/d,3,transporte de armas,40.740767,-73.995798
2,2022-01-25 10:00:00,2,s/d,2,comercio de pinguinos,40.753579,-74.000437
3,2022-01-25 14:00:00,3,s/d,2,incendio intencional,40.748401,-73.993990
4,2022-01-25 17:00:00,4,s/d,2,robo de vehiculo,40.743804,-73.992243
...,...,...,...,...,...,...,...
9995,2024-05-03 18:00:00,9995,s/d,1,comercio de pinguinos,40.737852,-73.992351
9996,2024-05-03 18:00:00,9996,s/d,2,hurto,40.739308,-74.000690
9997,2024-05-03 20:00:00,9997,s/d,2,robo,40.741509,-73.998346
9998,2024-05-03 23:00:00,9998,s/d,2,incendio intencional,40.738673,-73.999573


## **a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.**

Se considera los trimestres del año.

In [48]:
# Convertir la columna de fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

In [49]:
# Agrupamos por trimestre y categoria. 
df_resuelto = df[df['estado_caso'] == 2].groupby([df['fecha'].dt.to_period('Q'), 'categoria']).size()
df_resuelto

fecha   categoria            
2022Q1  comercio de pinguinos    77
        hurto                    58
        incendio intencional     62
        mercaderia ilegal        60
        robo                     81
                                 ..
2024Q2  incendio intencional     50
        mercaderia ilegal        45
        robo                     32
        robo de vehiculo         47
        transporte de armas      58
Length: 70, dtype: int64

In [50]:
df_total = df.groupby([df['fecha'].dt.to_period('Q'), 'categoria']).size()
df_total

fecha   categoria            
2022Q1  comercio de pinguinos    128
        hurto                     96
        incendio intencional     116
        mercaderia ilegal         99
        robo                     135
                                ... 
2024Q2  incendio intencional      69
        mercaderia ilegal         64
        robo                      46
        robo de vehiculo          65
        transporte de armas       70
Length: 70, dtype: int64

In [51]:
tasa_resolucion = df_resuelto / df_total
tasa_resolucion

fecha   categoria            
2022Q1  comercio de pinguinos    0.601562
        hurto                    0.604167
        incendio intencional     0.534483
        mercaderia ilegal        0.606061
        robo                     0.600000
                                   ...   
2024Q2  incendio intencional     0.724638
        mercaderia ilegal        0.703125
        robo                     0.695652
        robo de vehiculo         0.723077
        transporte de armas      0.828571
Length: 70, dtype: float64

In [52]:
tasa_resolucion_ultimo_trimestre = tasa_resolucion.loc['2024Q1':].groupby("categoria").transform(lambda x: x.pct_change())
tasa_resolucion_ultimo_trimestre

fecha   categoria            
2024Q1  comercio de pinguinos         NaN
        hurto                         NaN
        incendio intencional          NaN
        mercaderia ilegal             NaN
        robo                          NaN
        robo de vehiculo              NaN
        transporte de armas           NaN
2024Q2  comercio de pinguinos   -0.002634
        hurto                    0.115830
        incendio intencional    -0.056604
        mercaderia ilegal       -0.157462
        robo                    -0.106398
        robo de vehiculo        -0.089459
        transporte de armas      0.089418
dtype: float64

In [53]:
tasa_resolucion_ultimo_trimestre[(tasa_resolucion_ultimo_trimestre) >= 0.1]

fecha   categoria
2024Q2  hurto        0.11583
dtype: float64

## **Resolución 2**
Si se interpreta los ultimos 6 meses y no trimestre.

In [54]:
df["fecha"].iloc[-1]

Timestamp('2024-05-03 23:00:00')

Si el dataset estuvise sin orden por fechas

In [55]:
# fecha_fin - pd.DateOffset(months=6): Resta 6 meses a la fecha final
ultimos_6meses = df[(fecha_fin - pd.DateOffset(months=6) <= df.fecha) 
                    & (df.fecha <= fecha_fin)]
ultimos_6meses['trimestre'] = (fecha_fin - ultimos_6meses['fecha']) <= pd.Timedelta('90 days')

C:\Users\lufer\AppData\Local\Temp\ipykernel_5484\377761606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ultimos_6meses['trimestre'] = (fecha_fin - ultimos_6meses['fecha']) <= pd.Timedelta('90 days')


In [56]:
# Coerción a número, simplemente para que sea más sencillo manipularlo
ultimos_6meses['trimestre'] = 1 * ultimos_6meses['trimestre']

C:\Users\lufer\AppData\Local\Temp\ipykernel_5484\3788036758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ultimos_6meses['trimestre'] = 1 * ultimos_6meses['trimestre']


In [57]:
casos_agrupados = ultimos_6meses.groupby(['categoria','trimestre', 'estado_caso']).agg('size')
casos_agrupados

categoria              trimestre  estado_caso
comercio de pinguinos  0          1               31
                                  2              126
                                  3               19
                       1          1               22
                                  2              112
                                  3               23
hurto                  0          1               25
                                  2              118
                                  3               12
                       1          1               25
                                  2              119
                                  3               16
incendio intencional   0          1               23
                                  2              116
                                  3               20
                       1          1               18
                                  2              114
                                  3               16


In [58]:
casos_agrupados = casos_agrupados.unstack(level="estado_caso")
casos_agrupados

estado_caso                       1    2   3
categoria             trimestre             
comercio de pinguinos 0          31  126  19
                      1          22  112  23
hurto                 0          25  118  12
                      1          25  119  16
incendio intencional  0          23  116  20
                      1          18  114  16
mercaderia ilegal     0          21  102  14
                      1          25  121  10
robo                  0          25  124  21
                      1          25  114  10
robo de vehiculo      0          23  117  21
                      1          27  134  13
transporte de armas   0          21  100  15
                      1          17  128  13

In [59]:
casos_agrupados['resueltosporc'] = casos_agrupados[2] / (casos_agrupados[1] + casos_agrupados[2] + casos_agrupados[3])
casos_agrupados

estado_caso                       1    2   3  resueltosporc
categoria             trimestre                            
comercio de pinguinos 0          31  126  19       0.715909
                      1          22  112  23       0.713376
hurto                 0          25  118  12       0.761290
                      1          25  119  16       0.743750
incendio intencional  0          23  116  20       0.729560
                      1          18  114  16       0.770270
mercaderia ilegal     0          21  102  14       0.744526
                      1          25  121  10       0.775641
robo                  0          25  124  21       0.729412
                      1          25  114  10       0.765101
robo de vehiculo      0          23  117  21       0.726708
                      1          27  134  13       0.770115
transporte de armas   0          21  100  15       0.735294
                      1          17  128  13       0.810127

In [60]:
casos_agrupados = casos_agrupados.unstack(level="trimestre")
casos_agrupados

estado_caso             1        2        3     resueltosporc          
trimestre               0   1    0    1   0   1             0         1
categoria                                                              
comercio de pinguinos  31  22  126  112  19  23      0.715909  0.713376
hurto                  25  25  118  119  12  16      0.761290  0.743750
incendio intencional   23  18  116  114  20  16      0.729560  0.770270
mercaderia ilegal      21  25  102  121  14  10      0.744526  0.775641
robo                   25  25  124  114  21  10      0.729412  0.765101
robo de vehiculo       23  27  117  134  21  13      0.726708  0.770115
transporte de armas    21  17  100  128  15  13      0.735294  0.810127

In [61]:
casos_agrupados[casos_agrupados["resueltosporc"][1] >= (casos_agrupados["resueltosporc"][0] + 0.1)].index.values

array([], dtype=object)

## **b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).**